In [1]:
import boto3
import json

### Mistral 7B

Prompt template for Mistral: https://www.promptingguide.ai/models/mistral-7b

In [2]:
SYSTEM_PROMPT = """<s>[INST] You are an assistant helping persons to book medical reservations. You should answer in the same language as the person who 
needs help. First, the person will tell you what he needs. Then, you will be given options for the reservation. 
The options will be presented as a list: [Option A, Option B, ...]. Notice that the options are in French because the customers are in France.
You will have to rank the best 3 options from all the listed options, with respect to the needs of the person, or all of them if there are less than 3.
If not explitally asked, you must not justify your answer.
The format for the answer, that you must imperatively follow, including the '[' and ']', is: [best option, 2nd best option, 3rd best option]\n"""

In [3]:
user_message = "I have some headeache, I want to visit a doctor."

In [4]:
TASK_PROMPT = f"""Here is the message of the user:\n\n{user_message}\n\nWhich type of specialist should this person see?
Here is the list of options: 
[Chirurgien dermatologique, 
Laser, 
Médecin généraliste, 
Spécialiste en chirurgie plastique reconstructrice et esthétique]
Do not explain your answer.
Do not forget to answer in the following format, without any addition: [best option, 2nd best option, 3rd best option][/INST]
["""

In [5]:
message = ''.join([SYSTEM_PROMPT, TASK_PROMPT])
prompt = json.dumps(message)
print(message)

<s>[INST] You are an assistant helping persons to book medical reservations. You should answer in the same language as the person who 
needs help. First, the person will tell you what he needs. Then, you will be given options for the reservation. 
The options will be presented as a list: [Option A, Option B, ...]. Notice that the options are in French because the customers are in France.
You will have to rank the best 3 options from all the listed options, with respect to the needs of the person, or all of them if there are less than 3.
If not explitally asked, you must not justify your answer.
The format for the answer, that you must imperatively follow, including the '[' and ']', is: [best option, 2nd best option, 3rd best option]
Here is the message of the user:

I have some headeache, I want to visit a doctor.

Which type of specialist should this person see?
Here is the list of options: 
[Chirurgien dermatologique, 
Laser, 
Médecin généraliste, 
Spécialiste en chirurgie plastique 

In [6]:
bedrock = boto3.client(service_name="bedrock-runtime")

body = json.dumps({
    "prompt": prompt,
    "max_tokens": 512,
    "top_p": 0.8,
    "temperature": 0.5,
})

modelId = "mistral.mistral-7b-instruct-v0:2"

accept = "application/json"
contentType = "application/json"

response = bedrock.invoke_model(
    body=body,
    modelId=modelId,
    accept=accept,
    contentType=contentType
)

# Read and parse the response body
response_body = json.loads(response.get('body').read())
response_text = response_body["outputs"][0]["text"]

stop = "]"
# Check if the stop token is present in the generated text
if stop in response_text:
    # If the stop token is found, truncate or modify the response
    truncated_text = response_text.split(stop)[0]  # Cut off at the stop token
    print(truncated_text)
else:
    # If the stop token isn't found, print the entire generated text
    print(response_text)

Médecin généraliste", "Spécialiste en médecine interne", "Chirurgien dermatologique"


It invents "Spécialiste en médecine interne"...

### Claude 3.5 Haiku

In [7]:
SYSTEM_PROMPT = """You are an assistant helping persons to book medical reservations. You should answer in the same language as the person who 
needs help. First, the person will tell you what he needs. Then, you will be given options for the reservation. 
The options will be presented as a list: [Option A, Option B, ...]. Notice that the options are in French because the customers are in France.
You will have to rank the best 3 options from all the listed options, with respect to the needs of the person, or all of them if there are less than 3.
If not explitally asked, you must not justify your answer.
The format for the answer, that you must imperatively follow, including the '[' and ']', is: [best option, 2nd best option, 3rd best option]\n"""

In [8]:
user_message = "I have some headeache, I want to visit a doctor."

In [9]:
TASK_PROMPT = f"""Here is the message of the user:\n\n{user_message}\n\nWhich type of specialist should this person see?
Here is the list of options: 
[Chirurgien dermatologique, 
Laser, 
Médecin généraliste, 
Spécialiste en chirurgie plastique reconstructrice et esthétique]
Do not explain your answer.
Do not forget to answer in the following format, without any addition: [best option, 2nd best option, 3rd best option]"""

In [10]:
message = ''.join([SYSTEM_PROMPT, TASK_PROMPT])
prompt = json.dumps(message)
print(message)

You are an assistant helping persons to book medical reservations. You should answer in the same language as the person who 
needs help. First, the person will tell you what he needs. Then, you will be given options for the reservation. 
The options will be presented as a list: [Option A, Option B, ...]. Notice that the options are in French because the customers are in France.
You will have to rank the best 3 options from all the listed options, with respect to the needs of the person, or all of them if there are less than 3.
If not explitally asked, you must not justify your answer.
The format for the answer, that you must imperatively follow, including the '[' and ']', is: [best option, 2nd best option, 3rd best option]
Here is the message of the user:

I have some headeache, I want to visit a doctor.

Which type of specialist should this person see?
Here is the list of options: 
[Chirurgien dermatologique, 
Laser, 
Médecin généraliste, 
Spécialiste en chirurgie plastique reconstruc

In [11]:
bedrock = boto3.client(service_name="bedrock-runtime")
body = json.dumps({
  "max_tokens": 256,
  "messages": [{"role": "user", "content": message}],
  "anthropic_version": "bedrock-2023-05-31"
})

response = bedrock.invoke_model(body=body, modelId="anthropic.claude-3-5-haiku-20241022-v1:0")

response_body = json.loads(response.get("body").read())
response_text = response_body['content'][0]['text']
print(response_text)

[Médecin généraliste, Chirurgien dermatologique, Laser]
